In [ ]:
import numpy as np 
import pandas as pd
from sklearn.linear_model import LogisticRegression

train=pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-training.csv").fillna(0)
test=pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-test.csv").fillna(0)
y = train["SeriousDlqin2yrs"]
train.drop(["SeriousDlqin2yrs","Unnamed: 0"],axis=1,inplace=True)
test.drop(["SeriousDlqin2yrs","Unnamed: 0"],axis=1,inplace=True)
x = train.columns

In [ ]:
train['cl1'] = (train[x[6]]<0.5)&(train[x[0]]>0.599)&(train[x[2]]>0.5)&(train[x[0]]>1.003) 
train['cl2'] = (train[x[6]]<0.5)&(~train['cl1']) 
train['cl3'] = (train[x[6]]>0.5)&(train[x[6]]<1.5)
train['cl4'] = (train[x[6]]>1.5)
test['cl1'] = (test[x[6]]<0.5)&(test[x[0]]>0.599)&(test[x[2]]>0.5)&(test[x[0]]>1.003) 
test['cl2'] = (test[x[6]]<0.5)&(~test['cl1']) 
test['cl3'] = (test[x[6]]>0.5)&(test[x[6]]<1.5)
test['cl4'] = (test[x[6]]>1.5)

In [ ]:
[test['cl1'].sum(),test['cl2'].sum(),test['cl3'].sum(),test['cl4'].sum()]

In [ ]:
[train['cl1'].sum(),train['cl2'].sum(),train['cl3'].sum(),train['cl4'].sum()]

In [ ]:
classifiers = [] 
for thecluster in ['cl1','cl2','cl3','cl4']:
    ctrain = train[train[thecluster]==1][x[:10]]
    cy = y[train[thecluster]==1]
#     w1 = cy.sum()/len(cy)
#     print(w1)
    if thecluster!='cl2': 
        classifiers.append(LogisticRegression(random_state=0, 
                        class_weight='balanced').fit(ctrain, cy))
    else:
        classifiers.append(LogisticRegression(random_state=0, 
                        class_weight={0:0.1,1:0.9}).fit(ctrain, cy))

    

In [ ]:
submission = pd.read_csv("/kaggle/input/GiveMeSomeCredit/sampleEntry.csv")
submission['0'] = classifiers[0].predict_proba(test[x[:10]])[:,1]
submission['1'] = classifiers[1].predict_proba(test[x[:10]])[:,1]
submission['2'] = classifiers[2].predict_proba(test[x[:10]])[:,1]
submission['3'] = classifiers[3].predict_proba(test[x[:10]])[:,1]
submission['Probability'] = submission['0']*test['cl1'] + submission['1']*test['cl2'] + submission['2']*test['cl3'] + submission['3']*test['cl4'] 
 

In [ ]:
def fantasy(x):
    return x*x
model = LogisticRegression(random_state=0, 
                    class_weight={0:0.1,1:0.9}).fit(train[x[:10]], y)
submission = pd.read_csv("/kaggle/input/GiveMeSomeCredit/sampleEntry.csv")  
submission['Probability'] = model.predict_proba(test[x[:10]])[:,1]
submission['Probability'].apply(fantasy)

In [ ]:

submission[['Id','Probability']].to_csv("monster.csv",index=False)
submission['Probability'].hist(bins=20)

# Модели

In [ ]:
def chop(x):
    x = x.tolist()[0]
    coefficients = []
    for item in x:
        if np.abs(item)<0.001:
            coefficients.append(0.0)
        else:
            coefficients.append(round(item*10000)/10000)
    return coefficients        
            
for i in range(4):
    print(chop(classifiers[i].coef_))

In [ ]:
classifiers[1].coef_

In [ ]:
train.columns